# Wait for camera initiazation

In [8]:
"""
Enhanced Integrated Driver Security and Drowsiness Detection System with Calibration

This system combines:
0. Camera initialization and startup interface (NEW!)
1. Face recognition security verification with unauthorized user alarm (runs first)
2. Personal calibration phase (runs after security verification)
3. Driver drowsiness and yawning detection with adaptive thresholds

New Startup Features:
- Camera initialization with 3-second warmup period
- Professional startup interface with progress indicators
- User-controlled system start with SPACE button
- Pre-flight checklist and system readiness verification
- Enhanced user experience with guided startup process

New Security Features:
- Unauthorized user detection and alarm system
- Multiple alarm triggers for non-authorized faces
- Security breach logging with timestamps
- Automatic attempt counter and cooldown system
- Visual and audio alerts for security violations

New Calibration Features:
- Personal baseline measurement for each driver
- Adaptive thresholds based on individual characteristics
- Lighting condition adaptation
- Sensitivity adjustment options
- Profile saving and loading for each driver
- Real-time threshold refinement

Requirements:
1. Install packages: pip install opencv-python dlib numpy scipy pygame face-recognition
2. Download shape_predictor_68_face_landmarks.dat from http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
3. Create 'Images_sec' folder with authorized driver photos (e.g., Ahmed.jpg, Bill.jpeg, etc.)
4. Place your MP3 alert file "music.wav" in the same directory
5. Ensure webcam is connected and working

Usage:
- Run the script and wait for camera initialization (3 seconds)
- Press SPACE when "PRESS SPACE TO START" button appears
- Security verification (needs 2 consecutive verifications)
- 🚨 ALARM triggers if unauthorized user detected (not in Images_sec folder)
- Calibration phase (15 seconds of baseline measurement)
- Drowsiness monitoring with personalized thresholds

Controls:
- SPACE: Start system (after initialization)
- 'q': Quit system
- 'r': Reset/restart calibration
- 's': Adjust sensitivity levels
- 'i': Toggle calibration info display
"""

import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist
import pygame
import time
import sys
import face_recognition
import os
import datetime
import json
import statistics
from collections import deque


class EnhancedDriverSafetySystem:
    def __init__(self, security_images_path="Images_sec", mp3_filename="music.wav"):
        # Security system initialization
        self.security_images_path = security_images_path
        self.authorized_drivers = []
        self.driver_names = []
        self.load_authorized_drivers()
        
        # Drowsiness detection initialization
        self.detector = None
        self.predictor = None
        self.init_drowsiness_detector()
        
        # System state
        self.security_verified = False
        self.calibration_complete = False
        self.verified_driver_name = ""
        self.consecutive_verifications = 0
        self.last_verified_name = ""
        
        # System startup state
        self.system_initialized = False
        self.camera_warming_up = True
        self.warmup_start_time = None
        self.warmup_duration = 3  # seconds for camera warmup
        
        # Security alarm system
        self.unauthorized_attempts = 0
        self.max_unauthorized_attempts = 3
        self.last_unauthorized_time = 0
        self.unauthorized_cooldown = 5  # seconds between alarms
        self.security_breach_active = False
        
        # Audio setup
        self.mp3_file = mp3_filename
        self.setup_audio()
        
        # Facial landmarks
        self.LEFT_EYE = [36, 37, 38, 39, 40, 41]
        self.RIGHT_EYE = [42, 43, 44, 45, 46, 47]
        self.MOUTH = [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
        
        # Default thresholds (will be overridden by calibration)
        self.DEFAULT_EYE_AR_THRESH = 0.25
        self.DEFAULT_MOUTH_AR_THRESH = 0.7
        self.EYE_AR_CONSEC_FRAMES = 20
        self.YAWN_CONSEC_FRAMES = 15
        
        # Calibration parameters
        self.CALIBRATION_DURATION = 15  # seconds
        self.calibration_start_time = None
        self.calibration_ear_values = []
        self.calibration_mar_values = []
        
        # Adaptive thresholds
        self.baseline_ear = None
        self.baseline_mar = None
        self.adaptive_eye_thresh = None
        self.adaptive_mouth_thresh = None
        
        # Sensitivity settings (1=low, 2=medium, 3=high)
        self.sensitivity_level = 2
        self.sensitivity_multipliers = {
            1: {"eye": 0.6, "mouth": 1.4},    # Low sensitivity
            2: {"eye": 0.75, "mouth": 1.25},  # Medium sensitivity  
            3: {"eye": 0.85, "mouth": 1.1}   # High sensitivity
        }
        
        # Real-time adaptation
        self.recent_ear_values = deque(maxlen=100)
        self.recent_mar_values = deque(maxlen=100)
        self.adaptation_counter = 0
        
        # Lighting adaptation
        self.baseline_brightness = None
        self.current_brightness = None
        
        # Profile management
        self.profiles_file = "driver_profiles.json"
        self.driver_profiles = self.load_driver_profiles()
        
        # Counters for drowsiness detection
        self.eye_counter = 0
        self.yawn_counter = 0
        self.drowsy_alert = False
        self.yawn_alert = False
        
        # Calibration display
        self.show_calibration_info = True
    
    def load_authorized_drivers(self):
        """Load and encode authorized driver faces"""
        try:
            driver_list = os.listdir(self.security_images_path)
            print(f"Found driver images: {driver_list}")
            
            images = []
            for driver_file in driver_list:
                img_path = os.path.join(self.security_images_path, driver_file)
                current_img = cv2.imread(img_path)
                if current_img is not None:
                    images.append(current_img)
                    self.driver_names.append(os.path.splitext(driver_file)[0])
            
            print(f"Authorized drivers: {self.driver_names}")
            
            # Encode faces
            for img in images:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                encodings = face_recognition.face_encodings(img_rgb)
                if encodings:
                    self.authorized_drivers.append(encodings[0])
                else:
                    print("Warning: Could not encode face in one of the images")
            
            print(f"Successfully encoded {len(self.authorized_drivers)} driver faces")
            
        except Exception as e:
            print(f"Error loading authorized drivers: {e}")
            print("Make sure 'Images_sec' folder exists with driver photos")
    
    def init_drowsiness_detector(self):
        """Initialize dlib components for drowsiness detection"""
        try:
            self.detector = dlib.get_frontal_face_detector()
            self.predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
            print("Drowsiness detection system initialized")
        except Exception as e:
            print(f"Error initializing drowsiness detector: {e}")
            print("Make sure 'shape_predictor_68_face_landmarks.dat' is available")
    
    def setup_audio(self):
        """Setup audio system for alerts"""
        try:
            pygame.mixer.init()
            self.alert_sound = pygame.mixer.Sound(self.mp3_file)
            print(f"Audio system ready - Using: {self.mp3_file}")
        except Exception as e:
            print(f"Audio setup failed: {e}")
            print("Will use system beep as fallback")
            self.alert_sound = None
    
    def load_driver_profiles(self):
        """Load saved driver calibration profiles"""
        try:
            if os.path.exists(self.profiles_file):
                with open(self.profiles_file, 'r') as f:
                    profiles = json.load(f)
                print(f"Loaded {len(profiles)} driver profiles")
                return profiles
            else:
                print("No existing profiles found, will create new ones")
                return {}
        except Exception as e:
            print(f"Error loading profiles: {e}")
            return {}
    
    def save_driver_profiles(self):
        """Save driver calibration profiles"""
        try:
            with open(self.profiles_file, 'w') as f:
                json.dump(self.driver_profiles, f, indent=2)
            print("Driver profiles saved successfully")
        except Exception as e:
            print(f"Error saving profiles: {e}")
    
    def save_current_profile(self):
        """Save current driver's calibration data"""
        if self.verified_driver_name and self.baseline_ear and self.baseline_mar:
            profile_data = {
                "baseline_ear": self.baseline_ear,
                "baseline_mar": self.baseline_mar,
                "baseline_brightness": self.baseline_brightness,
                "sensitivity_level": self.sensitivity_level,
                "adaptive_eye_thresh": self.adaptive_eye_thresh,
                "adaptive_mouth_thresh": self.adaptive_mouth_thresh,
                "last_calibration": datetime.datetime.now().isoformat()
            }
            
            self.driver_profiles[self.verified_driver_name] = profile_data
            self.save_driver_profiles()
            print(f"Profile saved for {self.verified_driver_name}")
    
    def load_driver_profile(self, driver_name):
        """Load calibration data for specific driver"""
        if driver_name in self.driver_profiles:
            profile = self.driver_profiles[driver_name]
            self.baseline_ear = profile.get("baseline_ear")
            self.baseline_mar = profile.get("baseline_mar")
            self.baseline_brightness = profile.get("baseline_brightness")
            self.sensitivity_level = profile.get("sensitivity_level", 2)
            self.adaptive_eye_thresh = profile.get("adaptive_eye_thresh")
            self.adaptive_mouth_thresh = profile.get("adaptive_mouth_thresh")
            
            print(f"Loaded profile for {driver_name}")
            print(f"  Baseline EAR: {self.baseline_ear:.3f}")
            print(f"  Baseline MAR: {self.baseline_mar:.3f}")
            print(f"  Sensitivity: {self.sensitivity_level}")
            return True
        return False
    
    def calculate_brightness(self, frame):
        """Calculate average brightness of the frame"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        return np.mean(gray)
    
    def adjust_for_lighting(self):
        """Adjust thresholds based on lighting conditions"""
        if self.baseline_brightness and self.current_brightness:
            brightness_ratio = self.current_brightness / self.baseline_brightness
            
            # Adjust thresholds based on lighting changes
            if brightness_ratio < 0.7:  # Darker conditions
                lighting_factor = 0.9
            elif brightness_ratio > 1.3:  # Brighter conditions
                lighting_factor = 1.1
            else:
                lighting_factor = 1.0
            
            self.adaptive_eye_thresh *= lighting_factor
            self.adaptive_mouth_thresh *= lighting_factor
    
    def play_alert(self):
        """Play alert sound"""
        if self.alert_sound:
            self.alert_sound.play()
        else:
            print('\a')  # System beep
            sys.stdout.flush()
    
    def play_security_alarm(self):
        """Play security breach alarm - more intense than regular alert"""
        if self.alert_sound:
            # Play the alert sound multiple times for security breach
            for i in range(3):
                self.alert_sound.play()
                time.sleep(0.3)
        else:
            # Multiple system beeps for security alarm
            for i in range(5):
                print('\a')
                sys.stdout.flush()
                time.sleep(0.2)
        
        # Log security event
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"\n🚨 SECURITY ALERT [{timestamp}]: UNAUTHORIZED ACCESS ATTEMPT")
        print(f"Attempt #{self.unauthorized_attempts}")
    
    def check_unauthorized_access(self, face_encodings):
        """Check if detected faces are unauthorized and trigger alarm"""
        current_time = time.time()
        
        # Only check if we have face encodings but no matches
        if len(face_encodings) > 0:
            unauthorized_detected = True
            
            # Check each detected face against authorized drivers
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(self.authorized_drivers, face_encoding, tolerance=0.6)
                if any(matches):
                    unauthorized_detected = False
                    break
            
            # If unauthorized person detected
            if unauthorized_detected:
                # Cooldown check to prevent spam
                if current_time - self.last_unauthorized_time > self.unauthorized_cooldown:
                    self.unauthorized_attempts += 1
                    self.last_unauthorized_time = current_time
                    self.security_breach_active = True
                    
                    # Play security alarm
                    self.play_security_alarm()
                    
                    # Reset after max attempts reached
                    if self.unauthorized_attempts >= self.max_unauthorized_attempts:
                        print(f"⚠️  MAXIMUM UNAUTHORIZED ATTEMPTS REACHED ({self.max_unauthorized_attempts})")
                        print("Consider checking the security footage and authorized driver list.")
                        # Reset counter but keep logging
                        self.unauthorized_attempts = 0
                
                return True
            else:
                # Reset breach status if authorized user detected
                self.security_breach_active = False
        
        return False
    
    def security_verification(self, frame):
        """Handle security verification phase"""
        # Resize for faster processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        
        # Find faces and encodings
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        
        # Check for unauthorized access first
        unauthorized_detected = self.check_unauthorized_access(face_encodings)
        
        current_frame_verified = False
        current_verified_name = ""
        
        for face_encoding, face_location in zip(face_encodings, face_locations):
            # Compare with authorized drivers
            matches = face_recognition.compare_faces(self.authorized_drivers, face_encoding, tolerance=0.6)
            face_distances = face_recognition.face_distance(self.authorized_drivers, face_encoding)
            
            if len(face_distances) > 0:
                best_match_index = np.argmin(face_distances)
                
                if matches[best_match_index]:
                    name = self.driver_names[best_match_index].upper()
                    current_frame_verified = True
                    current_verified_name = name
                    
                    # Reset unauthorized attempts on successful verification
                    if self.unauthorized_attempts > 0:
                        print(f"✅ Authorized user {name} detected. Resetting security alerts.")
                        self.unauthorized_attempts = 0
                        self.security_breach_active = False
                    
                    # Check consecutive verifications
                    if name == self.last_verified_name:
                        self.consecutive_verifications += 1
                    else:
                        self.consecutive_verifications = 1
                        self.last_verified_name = name
                    
                    # Draw verification info
                    top, right, bottom, left = face_location
                    top *= 4
                    right *= 4
                    bottom *= 4
                    left *= 4
                    
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                    cv2.putText(frame, f"{name} VERIFIED", (left + 6, bottom - 6), 
                              cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)
                    
                    # Check if verification complete
                    if self.consecutive_verifications >= 2:
                        self.security_verified = True
                        self.verified_driver_name = name
                        return True
                else:
                    # Draw unauthorized user warning
                    top, right, bottom, left = face_location
                    top *= 4
                    right *= 4
                    bottom *= 4
                    left *= 4
                    
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 3)
                    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                    cv2.putText(frame, "UNAUTHORIZED", (left + 6, bottom - 6), 
                              cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2)
        
        # Reset if no verification in current frame
        if not current_frame_verified and self.consecutive_verifications > 0:
            self.consecutive_verifications = 0
            self.last_verified_name = ""
        
        # Display security breach warnings
        if self.security_breach_active or unauthorized_detected:
            # Flash red border
            cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 15)
            
            # Security breach message
            cv2.rectangle(frame, (50, 200), (590, 280), (0, 0, 255), cv2.FILLED)
            cv2.putText(frame, "🚨 SECURITY BREACH 🚨", (60, 230), 
                       cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(frame, "UNAUTHORIZED USER DETECTED", (60, 260), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            
            # Show attempt counter
            cv2.putText(frame, f"Unauthorized Attempts: {self.unauthorized_attempts}", (50, 300), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # Display verification progress for authorized users
        if self.consecutive_verifications > 0 and not unauthorized_detected:
            cv2.putText(frame, f"Verifications: {self.consecutive_verifications}/2", 
                       (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
        
        # Display instructions
        if not unauthorized_detected:
            cv2.putText(frame, "SECURITY VERIFICATION", (50, 100), 
                       cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, "Look directly at camera", (50, 130), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        else:
            cv2.putText(frame, "⛔ ACCESS DENIED ⛔", (50, 100), 
                       cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 3)
            cv2.putText(frame, "User not in authorized list", (50, 130), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, "Contact system administrator", (50, 160), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        return False
    
    def start_calibration(self):
        """Initialize calibration phase"""
        self.calibration_start_time = time.time()
        self.calibration_ear_values = []
        self.calibration_mar_values = []
        self.baseline_brightness = None
        print(f"\n=== STARTING CALIBRATION FOR {self.verified_driver_name} ===")
        print(f"Look straight ahead and blink normally for {self.CALIBRATION_DURATION} seconds")
    
    def calibration_phase(self, frame):
        """Handle calibration phase"""
        if self.calibration_start_time is None:
            self.start_calibration()
        
        current_time = time.time()
        elapsed_time = current_time - self.calibration_start_time
        remaining_time = self.CALIBRATION_DURATION - elapsed_time
        
        if remaining_time <= 0:
            # Calibration complete
            self.complete_calibration()
            return True
        
        # Process current frame for calibration
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.detector(gray, 0)
        
        # Calculate current brightness
        if self.baseline_brightness is None:
            self.baseline_brightness = self.calculate_brightness(frame)
        
        for face in faces:
            landmarks = self.extract_landmarks(gray, face)
            
            # Extract eye and mouth coordinates
            left_eye = landmarks[self.LEFT_EYE]
            right_eye = landmarks[self.RIGHT_EYE]
            mouth = landmarks[self.MOUTH]
            
            # Calculate aspect ratios
            left_ear = self.eye_aspect_ratio(left_eye)
            right_ear = self.eye_aspect_ratio(right_eye)
            ear = (left_ear + right_ear) / 2.0
            mar = self.mouth_aspect_ratio(mouth)
            
            # Store calibration values
            self.calibration_ear_values.append(ear)
            self.calibration_mar_values.append(mar)
            
            # Draw calibration interface
            cv2.drawContours(frame, [cv2.convexHull(left_eye)], -1, (255, 255, 0), 2)
            cv2.drawContours(frame, [cv2.convexHull(right_eye)], -1, (255, 255, 0), 2)
            cv2.drawContours(frame, [cv2.convexHull(mouth)], -1, (255, 255, 0), 2)
        
        # Display calibration progress
        progress = (elapsed_time / self.CALIBRATION_DURATION) * 100
        cv2.rectangle(frame, (50, 50), (590, 80), (0, 0, 0), cv2.FILLED)
        cv2.rectangle(frame, (50, 50), (50 + int(540 * progress / 100), 80), (0, 255, 0), cv2.FILLED)
        cv2.putText(frame, f"CALIBRATING: {remaining_time:.1f}s", (60, 70), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        # Instructions
        cv2.putText(frame, "Look straight ahead", (50, 120), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        cv2.putText(frame, "Blink naturally", (50, 150), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        cv2.putText(frame, "Keep steady lighting", (50, 180), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        return False
    
    def complete_calibration(self):
        """Complete calibration and set adaptive thresholds"""
        if len(self.calibration_ear_values) > 10 and len(self.calibration_mar_values) > 10:
            # Calculate baselines (remove outliers by using median)
            self.baseline_ear = statistics.median(self.calibration_ear_values)
            self.baseline_mar = statistics.median(self.calibration_mar_values)
            
            # Calculate standard deviations for adaptive thresholds
            ear_std = statistics.stdev(self.calibration_ear_values)
            mar_std = statistics.stdev(self.calibration_mar_values)
            
            # Set adaptive thresholds based on sensitivity level
            sensitivity = self.sensitivity_multipliers[self.sensitivity_level]
            
            # Eye threshold: baseline minus a factor based on sensitivity
            self.adaptive_eye_thresh = self.baseline_ear * sensitivity["eye"]
            
            # Mouth threshold: baseline plus a factor based on sensitivity  
            self.adaptive_mouth_thresh = self.baseline_mar * sensitivity["mouth"]
            
            # Ensure thresholds are within reasonable bounds
            self.adaptive_eye_thresh = max(0.15, min(0.35, self.adaptive_eye_thresh))
            self.adaptive_mouth_thresh = max(0.5, min(1.2, self.adaptive_mouth_thresh))
            
            self.calibration_complete = True
            
            # Save profile
            self.save_current_profile()
            
            print(f"\n=== CALIBRATION COMPLETE ===")
            print(f"Baseline EAR: {self.baseline_ear:.3f}")
            print(f"Baseline MAR: {self.baseline_mar:.3f}")
            print(f"Adaptive Eye Threshold: {self.adaptive_eye_thresh:.3f}")
            print(f"Adaptive Mouth Threshold: {self.adaptive_mouth_thresh:.3f}")
            print(f"Sensitivity Level: {self.sensitivity_level}")
            print("Starting drowsiness monitoring...")
            
        else:
            print("Calibration failed - insufficient data. Using default thresholds.")
            self.adaptive_eye_thresh = self.DEFAULT_EYE_AR_THRESH
            self.adaptive_mouth_thresh = self.DEFAULT_MOUTH_AR_THRESH
            self.calibration_complete = True
    
    def adjust_sensitivity(self):
        """Cycle through sensitivity levels"""
        self.sensitivity_level = (self.sensitivity_level % 3) + 1
        
        if self.baseline_ear and self.baseline_mar:
            # Recalculate thresholds with new sensitivity
            sensitivity = self.sensitivity_multipliers[self.sensitivity_level]
            self.adaptive_eye_thresh = self.baseline_ear * sensitivity["eye"]
            self.adaptive_mouth_thresh = self.baseline_mar * sensitivity["mouth"]
            
            # Ensure within bounds
            self.adaptive_eye_thresh = max(0.15, min(0.35, self.adaptive_eye_thresh))
            self.adaptive_mouth_thresh = max(0.5, min(1.2, self.adaptive_mouth_thresh))
        
        sensitivity_names = {1: "LOW", 2: "MEDIUM", 3: "HIGH"}
        print(f"Sensitivity changed to: {sensitivity_names[self.sensitivity_level]}")
        return sensitivity_names[self.sensitivity_level]
    
    def update_adaptive_thresholds(self, ear, mar):
        """Update thresholds based on recent normal behavior"""
        self.recent_ear_values.append(ear)
        self.recent_mar_values.append(mar)
        
        self.adaptation_counter += 1
        
        # Update baselines every 200 frames (if not in alert state)
        if self.adaptation_counter >= 200 and not self.drowsy_alert and not self.yawn_alert:
            if len(self.recent_ear_values) >= 50:
                # Update baseline as moving average
                recent_ear_median = statistics.median(list(self.recent_ear_values))
                recent_mar_median = statistics.median(list(self.recent_mar_values))
                
                # Gradual adaptation (blend with previous baseline)
                self.baseline_ear = 0.9 * self.baseline_ear + 0.1 * recent_ear_median
                self.baseline_mar = 0.9 * self.baseline_mar + 0.1 * recent_mar_median
                
                # Recalculate adaptive thresholds
                sensitivity = self.sensitivity_multipliers[self.sensitivity_level]
                self.adaptive_eye_thresh = self.baseline_ear * sensitivity["eye"]
                self.adaptive_mouth_thresh = self.baseline_mar * sensitivity["mouth"]
                
                # Ensure within bounds
                self.adaptive_eye_thresh = max(0.15, min(0.35, self.adaptive_eye_thresh))
                self.adaptive_mouth_thresh = max(0.5, min(1.2, self.adaptive_mouth_thresh))
            
            self.adaptation_counter = 0
    
    def eye_aspect_ratio(self, eye):
        """Calculate eye aspect ratio"""
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        C = dist.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)
        return ear
    
    def mouth_aspect_ratio(self, mouth):
        """Calculate mouth aspect ratio"""
        A = dist.euclidean(mouth[2], mouth[10])
        B = dist.euclidean(mouth[4], mouth[8])
        C = dist.euclidean(mouth[0], mouth[6])
        mar = (A + B) / (2.0 * C)
        return mar
    
    def extract_landmarks(self, gray, rect):
        """Extract facial landmarks"""
        shape = self.predictor(gray, rect)
        coords = np.zeros((68, 2), dtype="int")
        
        for i in range(0, 68):
            coords[i] = (shape.part(i).x, shape.part(i).y)
        
        return coords
    
    def drowsiness_monitoring(self, frame):
        """Handle drowsiness monitoring phase"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.detector(gray, 0)
        
        # Update current brightness for lighting adaptation
        self.current_brightness = self.calculate_brightness(frame)
        
        for face in faces:
            landmarks = self.extract_landmarks(gray, face)
            
            # Extract eye and mouth coordinates
            left_eye = landmarks[self.LEFT_EYE]
            right_eye = landmarks[self.RIGHT_EYE]
            mouth = landmarks[self.MOUTH]
            
            # Calculate aspect ratios
            left_ear = self.eye_aspect_ratio(left_eye)
            right_ear = self.eye_aspect_ratio(right_eye)
            ear = (left_ear + right_ear) / 2.0
            mar = self.mouth_aspect_ratio(mouth)
            
            # Update adaptive thresholds
            self.update_adaptive_thresholds(ear, mar)
            
            # Adjust for lighting conditions
            self.adjust_for_lighting()
            
            # Draw contours
            cv2.drawContours(frame, [cv2.convexHull(left_eye)], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [cv2.convexHull(right_eye)], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [cv2.convexHull(mouth)], -1, (0, 255, 0), 1)
            
            # Drowsiness detection with adaptive thresholds
            if ear < self.adaptive_eye_thresh:
                self.eye_counter += 1
                if self.eye_counter >= self.EYE_AR_CONSEC_FRAMES:
                    if not self.drowsy_alert:
                        self.drowsy_alert = True
                        self.play_alert()
            else:
                self.eye_counter = 0
                self.drowsy_alert = False
            
            # Yawning detection with adaptive thresholds
            if mar > self.adaptive_mouth_thresh:
                self.yawn_counter += 1
                if self.yawn_counter >= self.YAWN_CONSEC_FRAMES:
                    if not self.yawn_alert:
                        self.yawn_alert = True
                        self.play_alert()
            else:
                self.yawn_counter = 0
                self.yawn_alert = False
            
            # Display metrics and thresholds
            y_offset = 30
            cv2.putText(frame, f"Driver: {self.verified_driver_name}", (10, y_offset),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            
            y_offset += 25
            cv2.putText(frame, f"EAR: {ear:.3f} (Thresh: {self.adaptive_eye_thresh:.3f})", (10, y_offset),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            
            y_offset += 20
            cv2.putText(frame, f"MAR: {mar:.3f} (Thresh: {self.adaptive_mouth_thresh:.3f})", (10, y_offset),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            
            y_offset += 20
            sensitivity_names = {1: "LOW", 2: "MED", 3: "HIGH"}
            cv2.putText(frame, f"Sensitivity: {sensitivity_names[self.sensitivity_level]}", (10, y_offset),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
            
            # Show baseline info if requested
            if self.show_calibration_info:
                y_offset += 20
                cv2.putText(frame, f"Baseline EAR: {self.baseline_ear:.3f}", (10, y_offset),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
                y_offset += 15
                cv2.putText(frame, f"Baseline MAR: {self.baseline_mar:.3f}", (10, y_offset),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
            
            # Alert messages
            if self.drowsy_alert:
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 200),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 8)
            
            if self.yawn_alert:
                cv2.putText(frame, "YAWNING DETECTED!", (10, 240),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 165, 255), 3)
    
    def camera_initialization_phase(self, frame):
        """Handle camera initialization and startup phase"""
        current_time = time.time()
        
        # Start warmup timer if not started
        if self.warmup_start_time is None:
            self.warmup_start_time = current_time
        
        # Check if warmup is complete
        elapsed_warmup = current_time - self.warmup_start_time
        warmup_remaining = self.warmup_duration - elapsed_warmup
        
        if elapsed_warmup >= self.warmup_duration:
            self.camera_warming_up = False
        
        # Create startup interface
        overlay = frame.copy()
        
        # Main background
        cv2.rectangle(overlay, (50, 100), (590, 400), (40, 40, 40), cv2.FILLED)
        cv2.addWeighted(overlay, 0.8, frame, 0.2, 0, frame)
        
        # Title
        cv2.putText(frame, "DRIVER SAFETY SYSTEM", (80, 140), 
                   cv2.FONT_HERSHEY_COMPLEX, 1.2, (0, 255, 255), 3)
        cv2.putText(frame, "Enhanced Security & Monitoring", (90, 170), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        if self.camera_warming_up:
            # Camera warmup phase
            progress = (elapsed_warmup / self.warmup_duration) * 100
            
            # Warmup progress bar
            cv2.rectangle(frame, (80, 200), (560, 230), (100, 100, 100), cv2.FILLED)
            cv2.rectangle(frame, (80, 200), (80 + int(480 * progress / 100), 230), (0, 255, 0), cv2.FILLED)
            cv2.rectangle(frame, (80, 200), (560, 230), (255, 255, 255), 2)
            
            cv2.putText(frame, f"Camera Initializing... {warmup_remaining:.1f}s", (90, 220), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            # Instructions during warmup
            cv2.putText(frame, "Please wait while camera adjusts", (90, 260), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, "Position yourself in front of camera", (90, 290), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)
            cv2.putText(frame, "Ensure good lighting conditions", (90, 315), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)
            
        else:
            # Ready to start - show start button
            cv2.putText(frame, "✓ Camera Ready", (90, 220), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            
            # Start button
            button_color = (0, 200, 0) if not self.system_initialized else (100, 200, 100)
            cv2.rectangle(frame, (200, 250), (440, 300), button_color, cv2.FILLED)
            cv2.rectangle(frame, (200, 250), (440, 300), (255, 255, 255), 3)
            cv2.putText(frame, "PRESS SPACE TO START", (210, 280), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Pre-start checklist
            cv2.putText(frame, "✓ Position: Face visible in camera", (90, 330), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            cv2.putText(frame, "✓ Lighting: Adequate illumination", (90, 350), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            cv2.putText(frame, "✓ Authorized: Your photo in Images_sec", (90, 370), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        
        # System info footer
        cv2.putText(frame, f"Authorized Drivers: {len(self.driver_names)}", (10, frame.shape[0] - 60), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        cv2.putText(frame, f"Images Folder: {self.security_images_path}", (10, frame.shape[0] - 40), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        cv2.putText(frame, "Press 'q' to quit", (10, frame.shape[0] - 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return self.system_initialized
    
    def reset_calibration(self):
        """Reset calibration and start over"""
        self.calibration_complete = False
        self.calibration_start_time = None
        self.calibration_ear_values = []
        self.calibration_mar_values = []
        self.baseline_ear = None
        self.baseline_mar = None
        self.baseline_brightness = None
        print("Calibration reset. Starting new calibration...")
    
    def run_system(self):
        """Main system loop"""
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
        # Give camera a moment to initialize
        print("🔄 Initializing camera...")
        time.sleep(1)
        
        print("=== ENHANCED DRIVER SAFETY SYSTEM WITH CALIBRATION ===")
        print("Phase 0: Camera Initialization & Startup")
        print("Phase 1: Security Verification with Unauthorized User Detection")
        print("Phase 2: Personal Calibration")
        print("Phase 3: Adaptive Drowsiness Monitoring")
        print("\n🔒 Security Features:")
        print("  - Face recognition for authorized drivers")
        print("  - Unauthorized user alarm system")
        print("  - Security breach logging and alerts")
        print("  - Automatic reset after successful verification")
        print("\n🎮 Controls:")
        print("  SPACE - Start system (after camera initialization)")
        print("  'q' - Quit system")
        print("  'r' - Reset/restart calibration")
        print("  's' - Adjust sensitivity (Low/Medium/High)")
        print("  'i' - Toggle calibration info display")
        print(f"\n🔊 Alert sound: {self.mp3_file}")
        print(f"📁 Authorized drivers folder: {self.security_images_path}")
        print(f"⚠️  Unauthorized attempt limit: {self.max_unauthorized_attempts}")
        print(f"⏱️  Security alarm cooldown: {self.unauthorized_cooldown} seconds")
        print("\n🚀 Please wait for camera initialization...")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                print("❌ Error: Could not read from camera")
                break
            
            # Flip frame for mirror effect
            frame = cv2.flip(frame, 1)
            
            # Handle different phases
            if not self.system_initialized:
                # Phase 0: Camera initialization and startup
                startup_complete = self.camera_initialization_phase(frame)
                
                if startup_complete:
                    print("\n✅ System startup complete!")
                    print("🔐 Starting security verification...")
                
            elif not self.security_verified:
                # Phase 1: Security verification
                verification_complete = self.security_verification(frame)
                
                if verification_complete:
                    # Show success message
                    cv2.rectangle(frame, (50, 200), (600, 350), (0, 255, 0), cv2.FILLED)
                    cv2.putText(frame, "ACCESS GRANTED!", (80, 240), 
                               cv2.FONT_HERSHEY_COMPLEX, 1.2, (255, 255, 255), 3)
                    cv2.putText(frame, f"Welcome {self.verified_driver_name}", (80, 280), 
                               cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                    cv2.putText(frame, "Starting calibration...", (80, 320), 
                               cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)
                    cv2.imshow('Enhanced Driver Safety System', frame)
                    cv2.waitKey(2000)  # Show for 2 seconds
                    
                    print(f"\n=== SECURITY VERIFIED: {self.verified_driver_name} ===")
                    
                    # Check if we have a saved profile for this driver
                    if self.load_driver_profile(self.verified_driver_name):
                        print("Loaded existing calibration profile")
                        self.calibration_complete = True
                    else:
                        print("No existing profile found. Starting calibration...")
                    
            elif not self.calibration_complete:
                # Phase 2: Calibration
                calibration_complete = self.calibration_phase(frame)
                
                if calibration_complete:
                    # Show calibration complete message
                    cv2.rectangle(frame, (50, 200), (600, 300), (0, 255, 0), cv2.FILLED)
                    cv2.putText(frame, "CALIBRATION COMPLETE!", (80, 230), 
                               cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                    cv2.putText(frame, "Starting monitoring...", (80, 270), 
                               cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)
                    cv2.imshow('Enhanced Driver Safety System', frame)
                    cv2.waitKey(1500)  # Show for 1.5 seconds
            
            else:
                # Phase 3: Drowsiness monitoring
                self.drowsiness_monitoring(frame)
            
            # Common display elements (only show if system is initialized)
            if self.system_initialized:
                if self.calibration_complete:
                    cv2.putText(frame, "Controls: q=quit, r=recalibrate, s=sensitivity", 
                               (10, frame.shape[0] - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
                
                # System status
                if self.calibration_complete:
                    status_color = (0, 255, 0)
                    status_text = "STATUS: MONITORING"
                elif self.security_verified:
                    status_color = (0, 165, 255)
                    status_text = "STATUS: CALIBRATING"
                else:
                    status_color = (0, 0, 255)
                    status_text = "STATUS: VERIFYING"
                
                cv2.putText(frame, status_text, (10, frame.shape[0] - 20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, status_color, 2)
            
            cv2.imshow('Enhanced Driver Safety System', frame)
            
            # Handle key presses
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord(' ') and not self.camera_warming_up and not self.system_initialized:
                # Space bar to start system after camera warmup
                self.system_initialized = True
                print("🚀 System started by user!")
            elif key == ord('r') and self.security_verified:
                self.reset_calibration()
            elif key == ord('s') and self.calibration_complete:
                sensitivity_name = self.adjust_sensitivity()
                print(f"Sensitivity set to: {sensitivity_name}")
            elif key == ord('i') and self.system_initialized:
                self.show_calibration_info = not self.show_calibration_info
                print(f"Calibration info display: {'ON' if self.show_calibration_info else 'OFF'}")
        
        # Cleanup
        cap.release()
        cv2.destroyAllWindows()
        if self.alert_sound:
            pygame.mixer.quit()
        
        print("\n👋 Enhanced Driver Safety System terminated.")
        print("Thank you for using the system safely!")

def main():
    """Main function to run the enhanced system"""
    try:
        system = EnhancedDriverSafetySystem()
        system.run_system()
    except Exception as e:
        print(f"System error: {e}")
        print("Please check all requirements are installed and files are in place")

if __name__ == "__main__":
    main()

Found driver images: ['Bill.jpeg', 'Elon.jpeg']
Authorized drivers: ['Bill', 'Elon']
Successfully encoded 2 driver faces
Drowsiness detection system initialized
Audio system ready - Using: music.wav
Loaded 3 driver profiles
🔄 Initializing camera...
=== ENHANCED DRIVER SAFETY SYSTEM WITH CALIBRATION ===
Phase 0: Camera Initialization & Startup
Phase 1: Security Verification with Unauthorized User Detection
Phase 2: Personal Calibration
Phase 3: Adaptive Drowsiness Monitoring

🔒 Security Features:
  - Face recognition for authorized drivers
  - Unauthorized user alarm system
  - Security breach logging and alerts
  - Automatic reset after successful verification

🎮 Controls:
  SPACE - Start system (after camera initialization)
  'q' - Quit system
  'r' - Reset/restart calibration
  's' - Adjust sensitivity (Low/Medium/High)
  'i' - Toggle calibration info display

🔊 Alert sound: music.wav
📁 Authorized drivers folder: Images_sec
⚠️  Unauthorized attempt limit: 3
⏱️  Security alarm cooldow

In [5]:
import pyttsx3
# Initialize text-to-speech engine  
text_speecc = pyttsx3.init()
# Set properties before adding anything to speak
text_speecc.setProperty('rate', 175)  # Speed of speech
text_speecc.setProperty('volume', 1)  # Volume 0-1
voices = text_speecc.getProperty('voices')
text_speecc.setProperty('voice', voices[2].id)  # Set to default voice
# Speak a message
text_speecc.say("Please don't drive in such a condition")
text_speecc.say("Enhanced Driver Safety System is ready to use")
text_speecc.runAndWait()